Import and Set up

In [ ]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

import pandas as pd

from helpers.utils import CustomImageDatasetForGender

# Check device availability
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(f"Using device: {device}")


Define Model 1

In [ ]:
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            #256x344
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1), 
            #254x342
            nn.BatchNorm2d(32), 
            nn.ReLU(),

            nn.Conv2d(32, 32, kernel_size = 3, stride = 1), 
            #252x340
            nn.BatchNorm2d(32), 
            nn.ReLU(),
            nn.MaxPool2d(2), #252x340 -> 126x170

            #126x170
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            #124x168
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            #122x166
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            #120x164
            nn.MaxPool2d(2), #120x164 -> 60x82

            #60x82
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            #58x80
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            #56x78
            nn.MaxPool2d(2), #56x78 -> 28x39

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(76800, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2) # 0: female, 1: male
            # nn.Flatten(),
            # nn.Linear(44032, 512),  # Make sure this matches the flattened output size
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Linear(512, 2)  # 2 classes: 0 -> female, 1 -> male
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Helper function to convert labels to readable strings
def label_to_str(label): 
    if isinstance(label, int):   # 0 -> Female, 1 -> Male
        return "Male" if label == 1 else "Female"
    if torch.is_tensor(label):
        val = label.item() if label.dim() == 0 else label[0].item()
        return "Male" if val == 1 else "Female"
    if isinstance(label, bool):
        return "Male" if label else "Female"
    return str(label)

model = GenderCNN().to(device)
print("Model architecture:")
print(model)

Model 2

In [ ]:
device = torch.device("cpu")
model = GenderCNN().to(device)

class GenderCNN(nn.Module):
    def __init__(self):
        
        super(GenderCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(2), # 256x344 -> 128x172

            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2), # 128x172 -> 64x86

            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2), # 64x86 -> 32x43
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*32*43, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2) # 0: female, 1: male
            # nn.Flatten(),
            # nn.Linear(44032, 512),  # Make sure this matches the flattened output size
            # nn.ReLU(),
            # nn.Dropout(0.5),
            # nn.Linear(512, 2)  # 2 classes: 0 -> female, 1 -> male
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Helper function to convert labels to readable strings
def label_to_str(label): 
    if isinstance(label, int):   # 0 -> Female, 1 -> Male
        return "Male" if label == 1 else "Female"
    if torch.is_tensor(label):
        val = label.item() if label.dim() == 0 else label[0].item()
        return "Male" if val == 1 else "Female"
    if isinstance(label, bool):
        return "Male" if label else "Female"
    return str(label)

model = GenderCNN().to(device)
print("Model architecture:")
print(model)

In [ ]:
# Model 3

device = torch.device("cpu")
model = GenderCNN().to(device)

class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(16), 
            nn.ReLU(),
            nn.MaxPool2d(2), # 256x344 -> 128x172

            nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2), # 128x172 -> 64x86

            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2), # 64x86 -> 32x43
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 43, 128),
            nn.ReLU(),
            nn.Linear(128, 2) # 0: female, 1: male
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x
    
def label_to_str(label): 
    if isinstance(label, int):   # 0 -> Female, 1 -> Male
        return "Male" if label == 1 else "Female"
    if torch.is_tensor(label):
        val = label.item() if label.dim() == 0 else label[0].item()
        return "Male" if val == 1 else "Female"
    if isinstance(label, bool):
        return "Male" if label else "Female"
    return str(label)

model = GenderCNN().to(device)
print("Model architecture:")
print(model)

In [ ]:
# Model 4

device = torch.device("cpu")
model = GenderCNN().to(device)

class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, padding = 1),
            nn.ReLU(),
            ##nn.BatchNorm2d(16),
            nn.MaxPool2d(2), # 256x344 -> 128x172
            nn.Conv2d(16, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            ##nn.BatchNorm2d(32),
            nn.MaxPool2d(2) # 128x172 -> 64x86
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 64 * 86, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 2) # 0: female, 1: male
        )

    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)
    
def label_to_str(label): 
    if isinstance(label, int):   # 0 -> Female, 1 -> Male
        return "Male" if label == 1 else "Female"
    if torch.is_tensor(label):
        val = label.item() if label.dim() == 0 else label[0].item()
        return "Male" if val == 1 else "Female"
    if isinstance(label, bool):
        return "Male" if label else "Female"
    return str(label)

model = GenderCNN().to(device)
print("Model architecture:")
print(model)

In [ ]:
# Model 5

device = torch.device("cpu")
model = GenderCNN().to(device)

class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
        )

        self.dummy_input = torch.ones(1, 1, 256, 344)
        with torch.no_grad():
            self.flattened_size = self._get_flattened_size(self.dummy_input)

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_size, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 2) # 0: female, 1: male
        )

    def _get_flattened_size(self, x):
        x = self.conv_layers(x)
        return x.view(x.size(0), -1).size(1)


    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)
    
def label_to_str(label): 
    if isinstance(label, int):   # 0 -> Female, 1 -> Male
        return "Male" if label == 1 else "Female"
    if torch.is_tensor(label):
        val = label.item() if label.dim() == 0 else label[0].item()
        return "Male" if val == 1 else "Female"
    if isinstance(label, bool):
        return "Male" if label else "Female"
    return str(label)

model = GenderCNN().to(device)
print("Model architecture:")
print(model)

Create Dataset and Dataloaders

In [158]:
# Transforms
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

csv_file = "data/boneage-training-dataset.csv"
img_dir = "data/processed/training-set"

# Load the CSV
df = pd.read_csv(csv_file)

# Create dataset & subset
dataset = CustomImageDatasetForGender(root_dir=img_dir, labels=df, transform=transform)
subset_dataset = Subset(dataset, range(600))  # example subset of 600 items
#dataloader = DataLoader(subset_dataset, batch_size=16, shuffle=True)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
# Use this
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

test_csv = "data/boneage-test-dataset.csv"
test_img_dir = "data/processed/test-set"

test_df = pd.read_csv(test_csv)
test_dataset = CustomImageDatasetForGender(root_dir=test_img_dir, labels=test_df, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_samples = 0
    correct_predictions = 0

    for i, batch in enumerate(dataloader):
        images, _, genders, _ = batch
        images = images.to(device)
        genders = genders.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, genders)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        total_samples += images.size(0)

        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == genders).sum().item()

        if i % 10 == 9:  # Print every 10 batches
            avg_loss = running_loss / total_samples
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {avg_loss:.4f}")
            epoch_accuracy = 100 * correct_predictions / total_samples
            print(f"Epoch {epoch+1} training accuracy: {epoch_accuracy:.2f}%")
            running_loss = 0.0
            total_samples = 0
            correct_predictions = 0

    # After each epoch, evaluate on the test dataset
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_dataloader:
            images, genders, img_ids = batch
            images = images.to(device)
            genders = genders.long().to(device)

            outputs = model(images)
            probs = nn.Softmax(dim=1)(outputs)
            preds = torch.argmax(probs, dim=1)

            correct = (preds.cpu() == genders.cpu()).sum().item()
            total_correct += correct
            total_samples += images.size(0)

            # Logging predictions
            batch_pred_prob = probs.max(dim=1)[0]
            pred_labels = [label_to_str(p) for p in preds]
            true_labels = [label_to_str(g) for g in genders]

            print("Test Batch Predictions:", pred_labels)
            print("Test Batch True genders:", true_labels)
            print("Test Batch prediction probabilities:", batch_pred_prob.tolist())

    test_accuracy = 100 * total_correct / total_samples
    print(f"Epoch {epoch+1} test accuracy: {test_accuracy:.2f}%")

print("Finished Training")


In [ ]:
# Counting total number of Male/Female
from collections import Counter

def count_train_gender_samples(dataset):
    gender_counts = Counter()
    for _, _, genders, _ in dataset:
        gender_counts[genders.item()] += 1
    return gender_counts
    
def count_test_gender_samples(dataloader):
    gender_counts = Counter()
    for _, genders, _ in dataloader:
        gender_counts[genders.item()] += 1
    return gender_counts

# For training dataset
gender_counts_train = count_train_gender_samples(dataset)
print("Training dataset (gender count):", gender_counts_train)

# For sub-dataset
gender_counts_subset = count_train_gender_samples(subset_dataset)
print("Subset dataset (gender count):", gender_counts_subset)

# For testing dataset
gender_counts_test = count_test_gender_samples(test_dataset)
print("Test dataset (gender count):", gender_counts_test)

In [ ]:
# Train Model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_samples = 0
    correct_predictions = 0

    for i, batch in enumerate(dataloader):
        images, _, genders, _ = batch
        images = images.to(device)
        genders = genders.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, genders)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        total_samples += images.size(0)

        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == genders).sum().item()

        if i % 10 == 9:  # Print every 10 batches
            avg_loss = running_loss / total_samples
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {avg_loss:.4f}")
            epoch_accuracy = 100 * correct_predictions / total_samples
            print(f"Epoch {epoch+1} accuracy: {epoch_accuracy:.2f}%")
            running_loss = 0.0
            total_samples = 0
            correct_predictions = 0
        

print("Finished Training")

# epoch = 5   -> accuracy 50%
# epoch = 10  -> accuracy 56.5%
# epoch = 20  -> accuracy 50%


Evaluate on test set

In [ ]:
test_csv = "data/boneage-test-dataset.csv"
test_img_dir = "data/processed/test-set"

test_df = pd.read_csv(test_csv)
test_dataset = CustomImageDatasetForGender(root_dir=test_img_dir, labels=test_df, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_dataloader:
        images, genders, img_ids = batch  # ignoring age here if your Dataset returns it
        images = images.to(device)
        genders = genders.long().to(device)

        outputs = model(images)
        probs = nn.Softmax(dim=1)(outputs)
        preds = torch.argmax(probs, dim=1)

        correct = (preds.cpu() == genders.cpu()).sum().item()
        total_correct += correct
        total_samples += images.size(0)

        # Logging predictions
        batch_pred_prob = probs.max(dim=1)[0]
        pred_labels = [label_to_str(p) for p in preds]
        true_labels = [label_to_str(g) for g in genders]

        print("Batch Predictions:", pred_labels)
        print("Batch True genders:", true_labels)
        print("Batch prediction probabilities:", batch_pred_prob.tolist())

overall_accuracy = total_correct / total_samples * 100
print(f"Overall accuracy on {total_samples} samples: {overall_accuracy:.2f}%")
